_한국어로 기계번역됨_


# 도구에 런타임 값을 전달하는 방법

가끔 도구를 호출하는 LLM이 도구 함수의 인수 중 일부를 채우고 나머지 값들은 런타임에 제공하도록 하고 싶을 때가 있습니다. LangChain 스타일의 [도구](https://python.langchain.com/docs/concepts/#tools)를 사용하고 있다면, 함수 매개변수를 [InjectedArg](https://python.langchain.com/docs/how_to/tool_runtime/)로 주석 처리하는 것이 간단한 방법입니다. 이 주석은 해당 매개변수를 LLM에 표시되지 않도록 배제합니다.

LangGraph 애플리케이션에서는 런타임에 도구에 그래프 상태 또는 [공유 메모리](https://langchain-ai.github.io/langgraph/how-tos/cross-thread-persistence/) (저장소)를 전달하고 싶을 수 있습니다. 이런 종류의 상태를 가진 도구는 도구의 출력이 과거 에이전트 단계에 의해 영향을 받을 때 유용합니다 (예: 도구로서 서브 에이전트를 사용하는 경우 메시지 이력을 서브 에이전트에 전달하고자 할 때) 또는 도구의 입력이 과거 에이전트 단계의 컨텍스트를 고려하여 검증되어야 할 때 유용합니다.

이 안내서에서는 LangGraph의 미리 구축된 [ToolNode](https://langchain-ai.github.io/langgraph/how-tos/tool-calling/)를 사용하여 이를 수행하는 방법을 보여줍니다.

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 안내서는 **LangChain 도구 호출**을 대상으로 하며 다음과 같은 내용을 친숙하게 알고 있어야 합니다:
        <ul>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#tools">
                    도구
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#state">
                    상태
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent">
                    도구 호출
                </a>
            </li>
        </ul>
        LangGraph의 핵심 기능을 손실하지 않고도 제공자 SDK를 사용하여 도구 호출을 수행할 수 있습니다.
    </p>
</div>

다음 예제의 핵심 기술은 매개변수를 "주입된" 것으로 주석 처리하는 것입니다. 이는 해당 매개변수가 프로그램에 의해 주입될 것이며 LLM에 의해 보이거나 채워지지 않아야 함을 의미합니다. 다음 코드 스니펫을 tl;dr로 활용해 보세요:

```python
from typing import Annotated

from langchain_core.runnables import RunnableConfig
from langchain_core.tools import InjectedToolArg
from langgraph.store.base import BaseStore

from langgraph.prebuilt import InjectedState, InjectedStore


# 동기 또는 비동기 모두 가능; @tool 데코레이터는 필요 없음
async def my_tool(
    # 이 인수들은 LLM에 의해 채워집니다
    some_arg: str,
    another_arg: float,
    # config: RunnableConfig는 항상 LangChain 호출에서 사용 가능합니다
    # 이는 LLM에 노출되지 않습니다
    config: RunnableConfig,
    # 다음 세 가지는 미리 구축된 ToolNode에 특화된 것입니다
    # (그리고 `create_react_agent`로 확장됩니다). 도구를 독립적으로 호출하는 경우
    # 이를 직접 제공해야 합니다.
    store: Annotated[BaseStore, InjectedStore],
    # 전체 상태를 전달합니다.
    state: Annotated[State, InjectedState],
    # 상태에서 단일 필드를 주입할 수도 있습니다.
    messages: Annotated[list, InjectedState("messages")]
    # 아래는 create_react_agent 또는 ToolNode와 호환되지 않습니다
    # 또한 모델에 표시되지 않도록 다른 인수를 배제할 수도 있습니다.
    # 이러한 인수는 수동으로 제공되어야 하며, 자신만의 노드에서 도구/함수를 호출할 때 유용합니다.
    # some_other_arg=Annotated["MyPrivateClass", InjectedToolArg],
):
    """my_tool을 호출하여 실제 세계에 영향을 미칩니다.

    Args:
        some_arg: 매우 중요한 인수
        another_arg: LLM이 제공할 다른 인수
    """ # 도큐먼트 문자열은 도구에 대한 설명이 되어 모델에 전달됩니다
    print(some_arg, another_arg, config, store, state, messages)
    # Config, some_other_rag, store 및 state는 모두 "숨겨진" 상태입니다.
    # bind_tools 또는 with_structured_output에 전달될 때 LangChain 모델에서
    return "... some response"
```


## 설정

먼저 필요한 패키지를 설치해야 합니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-openai


다음으로, 사용하려는 채팅 모델인 OpenAI의 API 키를 설정해야 합니다.


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


OPENAI_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 확인하고 성능을 개선하세요. LangSmith는 LangGraph로 구축한 LLM 앱을 디버그하고 테스트하며 모니터링할 수 있도록 추적 데이터를 제공합니다 — <a href="https://docs.smith.langchain.com">여기</a>에서 시작하는 방법에 대해 자세히 알아보세요.
    </p>
</div>


## 그래프 상태를 도구에 전달하기

먼저 도구에 그래프 상태에 대한 접근을 제공하는 방법을 살펴보겠습니다. 그래프 상태를 정의해야 합니다:


In [3]:
from typing import List

# this is the state schema used by the prebuilt create_react_agent we'll be using below
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.documents import Document


class State(AgentState):
    docs: List[str]


### 도구 정의하기

우리는 도구가 그래프 상태를 입력으로 받도록 하고 싶지만, 도구를 호출할 때 모델이 이 입력을 생성하지 않도록 하고 싶습니다. `InjectedState` 주석을 사용하여 인수를 필수 그래프 상태(또는 그래프 상태의 일부 필드)로 표시할 수 있습니다. 이러한 인수는 모델에 의해 생성되지 않습니다. `ToolNode`를 사용할 때, 그래프 상태는 관련 도구와 인수에 자동으로 전달됩니다.

이 예제에서는 문서를 반환하는 도구와 주장을 정당화하는 문서를 실제로 인용하는 또 다른 도구를 만들 것입니다.


<div class="admonition note">
    <p class="admonition-title">LangChain과 Pydantic 사용하기</p>
    <p>
        이 노트북은 Pydantic v2 <code>BaseModel</code>을 사용하며, <code>langchain-core >= 0.3</code>이 필요합니다. <code>langchain-core < 0.3</code>을 사용하는 경우 Pydantic v1과 v2 <code>BaseModels</code>의 혼합으로 인해 오류가 발생합니다.
    </p>
</div>


In [4]:
from typing import List, Tuple
from typing_extensions import Annotated

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState


@tool
def get_context(question: str, state: Annotated[dict, InjectedState]):
    """Get relevant context for answering the question."""
    return "\n\n".join(doc for doc in state["docs"])


이 도구들의 입력 스키마를 살펴보면 `state`가 여전히 나열되어 있는 것을 볼 수 있습니다:


In [5]:
get_context.get_input_schema().schema()


{'description': 'Get relevant context for answering the question.',
 'properties': {'question': {'title': 'Question', 'type': 'string'},
  'state': {'title': 'State', 'type': 'object'}},
 'required': ['question', 'state'],
 'title': 'get_context',
 'type': 'object'}

하지만 도구 호출을 위한 모델에 전달되는 도구 호출 스키마를 살펴보면, `state`가 제거되었습니다:


In [6]:
get_context.tool_call_schema.schema()


{'description': 'Get relevant context for answering the question.',
 'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'get_context',
 'type': 'object'}

### 그래프 정의

이번 예제에서는 [사전 구축된 ReAct 에이전트](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/)를 사용할 것입니다. 먼저 모델과 도구 호출 노드 ([ToolNode](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.tool_node.ToolNode))를 정의해야 합니다:


In [7]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver

model = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [get_context]

# ToolNode will automatically take care of injecting state into tools
tool_node = ToolNode(tools)

checkpointer = MemorySaver()
graph = create_react_agent(model, tools, state_schema=State, checkpointer=checkpointer)


### 사용해보세요!


In [8]:
docs = [
    "FooBar company just raised 1 Billion dollars!",
    "FooBar company was founded in 2019",
]

inputs = {
    "messages": [{"type": "user", "content": "what's the latest news about FooBar"}],
    "docs": docs,
}
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream(inputs, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's the latest news about FooBar
================================== Ai Message ==================================
Tool Calls:
  get_context (call_UkqfR7z2cLJQjhatUpDeEa5H)
 Call ID: call_UkqfR7z2cLJQjhatUpDeEa5H
  Args:
    question: latest news about FooBar
================================= Tool Message =================================
Name: get_context

FooBar company just raised 1 Billion dollars!

FooBar company was founded in 2019
================================== Ai Message ==================================

The latest news about FooBar is that the company has just raised 1 billion dollars.


## 그래프에 공유 메모리(저장소) 전달

여러 대화나 사용자 간에 공유되는 메모리에 도구가 접근할 수 있도록 하고 싶을 수 있습니다. 이를 위해 LangGraph [Store](https://langchain-ai.github.io/langgraph/how-tos/cross-thread-persistence/)를 도구에 전달하면서 다른 주석 `InjectedStore`를 사용할 수 있습니다.

예제를 수정하여 문서를 메모리 내 저장소에 저장하고 `get_context` 도구를 사용하여 이를 검색하도록 하겠습니다. 또한 사용자 ID에 따라 문서에 접근할 수 있도록 하여 특정 사용자에게만 표시되는 문서가 있도록 하겠습니다. 도구는 [config](https://langchain-ai.github.io/langgraph/how-tos/pass-config-to-tools/)에 제공된 `user_id`를 사용하여 올바른 문서 세트를 검색합니다.


<div class="admonition note">
    <p class="admonition-title">참고</p>
    <list>
        <li>
        이 노트북에서 사용된 <code>Store</code> API 및 <code>InjectedStore</code>에 대한 지원은 LangGraph <code>v0.2.34</code>에서 추가되었습니다.
        </li>
        <li>
        <code>InjectedStore</code> 주석은 <code>langchain-core >= 0.3.8</code>을 필요로 합니다.
        </li>
    <list>
</div>


In [9]:
from langgraph.store.memory import InMemoryStore

doc_store = InMemoryStore()

namespace = ("documents", "1")  # user ID
doc_store.put(
    namespace, "doc_0", {"doc": "FooBar company just raised 1 Billion dollars!"}
)
namespace = ("documents", "2")  # user ID
doc_store.put(namespace, "doc_1", {"doc": "FooBar company was founded in 2019"})


### 도구 정의하기


In [10]:
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import InjectedStore


@tool
def get_context(
    question: str,
    config: RunnableConfig,
    store: Annotated[BaseStore, InjectedStore()],
) -> Tuple[str, List[Document]]:
    """Get relevant context for answering the question."""
    user_id = config.get("configurable", {}).get("user_id")
    docs = [item.value["doc"] for item in store.search(("documents", user_id))]
    return "\n\n".join(doc for doc in docs)


우리는 또한 도구 호출 모델이 `get_context` 도구의 `store` 인자를 무시할 것임을 확인할 수 있습니다:


In [11]:
get_context.tool_call_schema.schema()


{'description': 'Get relevant context for answering the question.',
 'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'get_context',
 'type': 'object'}

### 그래프 정의

우리 ReAct 에이전트를 업데이트합시다:


In [12]:
tools = [get_context]

# ToolNode will automatically take care of injecting Store into tools
tool_node = ToolNode(tools)

checkpointer = MemorySaver()
# NOTE: we need to pass our store to `create_react_agent` to make sure our graph is aware of it
graph = create_react_agent(model, tools, checkpointer=checkpointer, store=doc_store)


### 사용하세요!


구성에서 `"user_id"`를 사용하여 그래프를 실행해 보겠습니다.


In [13]:
messages = [{"type": "user", "content": "what's the latest news about FooBar"}]
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
for chunk in graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's the latest news about FooBar
================================== Ai Message ==================================
Tool Calls:
  get_context (call_ocyHBpGgF3LPFOgRKURBfkGG)
 Call ID: call_ocyHBpGgF3LPFOgRKURBfkGG
  Args:
    question: latest news about FooBar
================================= Tool Message =================================
Name: get_context

FooBar company just raised 1 Billion dollars!
================================== Ai Message ==================================

The latest news about FooBar is that the company has just raised 1 billion dollars.


우리는 도구가 상점에서 정보를 조회할 때 사용자 "1"에 대한 올바른 문서만 검색한 것을 볼 수 있습니다. 이제 다른 사용자에 대해 다시 시도해 보겠습니다:


In [14]:
messages = [{"type": "user", "content": "what's the latest news about FooBar"}]
config = {"configurable": {"thread_id": "2", "user_id": "2"}}
for chunk in graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what's the latest news about FooBar
================================== Ai Message ==================================
Tool Calls:
  get_context (call_zxO9KVlL8UxFQUMb8ETeHNvs)
 Call ID: call_zxO9KVlL8UxFQUMb8ETeHNvs
  Args:
    question: latest news about FooBar
================================= Tool Message =================================
Name: get_context

FooBar company was founded in 2019
================================== Ai Message ==================================

FooBar company was founded in 2019. If you need more specific or recent news, please let me know!


이번에는 도구가 다른 문서를 가져온 것을 볼 수 있습니다.
